# List of Exposures with Hologram


- work with Weakly_2022_09
- use jupyter kernel LSST



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/03/22

- update : 2022/05/05
- last update : 2022/05/19 : check if 2022/05/04 where already ingested
- last update : 2022/09/13


In [1]:
import sys
print(sys.executable)

/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/bin/python


In [2]:
print(sys.path)

['/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/analysis_tools/gb9b489fd42+90dca76f06/python', '/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/analysis_drp/g12d5fbe6a5+90dca76f06/python', '/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/spectractor/g90263d33d7/lib/python', '/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/cp_verify/g41175167c7+16959348af/python', '/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/faro/gbb36616a4b+5e2d81108f/python', '/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/fgcm/g3dbc634184/lib/python', '/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/fgcmcal/g5147cf65fe+deeae05922/py

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/drp_pipe 	setup
eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/eups 	setup
rapid_analysis        LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/rapid_analysis 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


   g0b29ad24fb+a10408d0bf 	w_latest current w_2022_39 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
import lsst.daf.butler as dafButler

In [8]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [9]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [10]:
for count, info in enumerate(registry.queryDimensionRecords('exposure')):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [11]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2021021600208,AT_O_20210216_000208,20210216,208,science,HD 50896,empty~empty,24.314073,2.0,103.384002,-23.973448,-180.0
1,2021021600163,AT_O_20210216_000163,20210216,163,science,HD 50896,RG610~empty,14.580611,3.0,103.404029,-23.92939,-208.621712
2,2021021600070,AT_O_20210216_000070,20210216,70,bias,azel_target,empty~empty,None,0.0,None,None,None
3,2021021600173,AT_O_20210216_000173,20210216,173,science,HD 50896,RG610~empty,8.82931,2.0,103.395986,-23.927038,-208.621712
4,2021021600001,AT_O_20210216_000001,20210216,1,bias,azel_target,empty~empty,None,0.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
21152,2022063000801,AT_O_20220630_000801,20220630,801,science,AUXTEL_DRP_IMAGING:WFI-2026-4536_062,SDSSr~empty,36.396484,30.0,306.677395,-45.325215,359.998841
21153,2022063000802,AT_O_20220630_000802,20220630,802,science,AUXTEL_DRP_IMAGING:WFI-2026-4536_063,SDSSr~empty,36.437127,30.0,306.811224,-45.325305,0.005404
21154,2022063000803,AT_O_20220630_000803,20220630,803,science,AUXTEL_DRP_IMAGING:WFI-2026-4536_063,SDSSr~empty,36.530061,30.0,306.811146,-45.325195,359.99891
21155,2022063000804,AT_O_20220630_000804,20220630,804,science,AUXTEL_DRP_IMAGING:WFI-2026-4536_064,SDSSr~empty,36.57364,30.0,306.944962,-45.325294,359.998843


## Selection of science

In [12]:
df_science = df_exposure[df_exposure.type == 'science']

In [13]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/scratch/users/d/dagoret/ipykernel_19563/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [14]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
21092,2022063000741,AT_O_20220630_000741,20220630,741,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.216657,30.0,309.341775,-5.594321,50.470185
21093,2022063000742,AT_O_20220630_000742,20220630,742,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.311557,30.0,309.341675,-5.594353,50.472142
21094,2022063000743,AT_O_20220630_000743,20220630,743,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.36748,30.0,309.436196,-5.594359,50.230394
21095,2022063000744,AT_O_20220630_000744,20220630,744,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.459853,30.0,309.436102,-5.594346,50.231639
21096,2022063000745,AT_O_20220630_000745,20220630,745,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.571891,30.0,328.698147,17.412737,83.126668
21097,2022063000746,AT_O_20220630_000746,20220630,746,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.597445,30.0,328.698026,17.412692,83.124157
21098,2022063000747,AT_O_20220630_000747,20220630,747,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.759167,30.0,328.50127,17.506591,82.501275
21099,2022063000748,AT_O_20220630_000748,20220630,748,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.786121,30.0,328.501103,17.506714,82.502325
21100,2022063000749,AT_O_20220630_000749,20220630,749,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.801233,30.0,328.599795,17.506644,82.299976
21101,2022063000750,AT_O_20220630_000750,20220630,750,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.82869,30.0,328.599615,17.506728,82.298997


In [15]:
df_science.day_obs.unique()

array([20210216, 20210217, 20210218, 20210311, 20210608, 20210609,
       20210610, 20210706, 20210707, 20210708, 20210907, 20210908,
       20210909, 20211005, 20211006, 20211102, 20211103, 20211104,
       20220215, 20220216, 20220217, 20220316, 20220317, 20220503,
       20220504, 20220524, 20220607, 20220608, 20220609, 20220628,
       20220629, 20220630])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [16]:
date_sel = 20220630

In [17]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [18]:
len(df_science_selected)

558

# List of filters

In [19]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'BG40~holo4_003', 'FELH0600~holo4_003',
       'empty~ronchi170lpmm', 'BG40~ronchi170lpmm',
       'FELH0600~ronchi170lpmm', 'SDSSr~empty'], dtype=object)

In [20]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003', 'BG40~holo4_003', 'FELH0600~holo4_003'],
      dtype='<U18')

## Selection of filter

- But now we have a loop on filters

In [21]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [22]:
pd.set_option('display.max_rows', None)

In [23]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20220630_filt_empty-holo4_003.list
BG40~holo4_003
visitdispersers_20220630_filt_BG40-holo4_003.list
FELH0600~holo4_003
visitdispersers_20220630_filt_FELH0600-holo4_003.list


/scratch/users/d/dagoret/ipykernel_19563/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/scratch/users/d/dagoret/ipykernel_19563/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/scratch/users/d/dagoret/ipykernel_19563/3368292521.py:20: SettingWithCopyWarning: 
A value is tryin

In [24]:
len(all_def_science_selected_final)

3

In [25]:
all_def_science_selected_final[0]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
19835,2022063000045,AT_O_20220630_000045,20220630,45,science,spec:HD142331,empty~holo4_003,44.7819,30.0,238.597624,-8.619684,145.262022
19836,2022063000046,AT_O_20220630_000046,20220630,46,science,spec:HD142331,empty~holo4_003,44.635934,30.0,238.597606,-8.619646,145.261566
20411,2022063000060,AT_O_20220630_000060,20220630,60,science,spec:HD142331,empty~holo4_003,42.779127,30.0,238.599505,-8.620962,144.104081
20412,2022063000061,AT_O_20220630_000061,20220630,61,science,spec:HD142331,empty~holo4_003,42.635099,30.0,238.599486,-8.62097,144.104102
20427,2022063000076,AT_O_20220630_000076,20220630,76,science,spec:HD185975,empty~holo4_003,60.4983,30.0,307.139255,-87.516853,162.963716
20428,2022063000077,AT_O_20220630_000077,20220630,77,science,spec:HD185975,empty~holo4_003,60.490649,30.0,307.1377,-87.516873,162.963109
20442,2022063000091,AT_O_20220630_000091,20220630,91,science,spec:HD142331,empty~holo4_003,38.374764,30.0,238.601248,-8.620652,141.191952
20443,2022063000092,AT_O_20220630_000092,20220630,92,science,spec:HD142331,empty~holo4_003,38.236606,30.0,238.601094,-8.62034,141.191566
20457,2022063000106,AT_O_20220630_000106,20220630,106,science,spec:HD142331,empty~holo4_003,36.490526,30.0,238.602593,-8.619824,139.344987
20458,2022063000107,AT_O_20220630_000107,20220630,107,science,spec:HD142331,empty~holo4_003,36.355539,30.0,238.602517,-8.619824,139.345002


In [26]:
all_def_science_selected_final[1]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
19837,2022063000047,AT_O_20220630_000047,20220630,47,science,spec:HD142331,BG40~holo4_003,44.476529,30.0,238.597621,-8.61967,145.261569
19838,2022063000048,AT_O_20220630_000048,20220630,48,science,spec:HD142331,BG40~holo4_003,44.334212,30.0,238.597599,-8.619655,145.261571
20413,2022063000062,AT_O_20220630_000062,20220630,62,science,spec:HD142331,BG40~holo4_003,42.484563,30.0,238.599494,-8.620975,144.104278
20414,2022063000063,AT_O_20220630_000063,20220630,63,science,spec:HD142331,BG40~holo4_003,42.340928,30.0,238.599514,-8.62091,144.104169
20429,2022063000078,AT_O_20220630_000078,20220630,78,science,spec:HD185975,BG40~holo4_003,60.482509,30.0,307.137198,-87.516871,162.963303
20430,2022063000079,AT_O_20220630_000079,20220630,79,science,spec:HD185975,BG40~holo4_003,60.474734,30.0,307.137214,-87.516875,162.964278
20444,2022063000093,AT_O_20220630_000093,20220630,93,science,spec:HD142331,BG40~holo4_003,38.088915,30.0,238.601048,-8.620379,141.191767
20445,2022063000094,AT_O_20220630_000094,20220630,94,science,spec:HD142331,BG40~holo4_003,37.9511,30.0,238.601032,-8.62043,141.191556
20459,2022063000108,AT_O_20220630_000108,20220630,108,science,spec:HD142331,BG40~holo4_003,36.208452,30.0,238.602622,-8.619797,139.345048
20460,2022063000109,AT_O_20220630_000109,20220630,109,science,spec:HD142331,BG40~holo4_003,36.077178,30.0,238.60257,-8.619762,139.344915


In [27]:
all_def_science_selected_final[2]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
20400,2022063000049,AT_O_20220630_000049,20220630,49,science,spec:HD142331,FELH0600~holo4_003,44.181785,30.0,238.597515,-8.619709,145.261545
20401,2022063000050,AT_O_20220630_000050,20220630,50,science,spec:HD142331,FELH0600~holo4_003,44.036499,30.0,238.597629,-8.619638,145.261701
20415,2022063000064,AT_O_20220630_000064,20220630,64,science,spec:HD142331,FELH0600~holo4_003,42.190713,30.0,238.599506,-8.620921,144.104262
20416,2022063000065,AT_O_20220630_000065,20220630,65,science,spec:HD142331,FELH0600~holo4_003,42.04734,30.0,238.599505,-8.620938,144.104228
20431,2022063000080,AT_O_20220630_000080,20220630,80,science,spec:HD185975,FELH0600~holo4_003,60.466945,30.0,307.136986,-87.516859,162.963418
20432,2022063000081,AT_O_20220630_000081,20220630,81,science,spec:HD185975,FELH0600~holo4_003,60.459332,30.0,307.137184,-87.516884,162.964405
20446,2022063000095,AT_O_20220630_000095,20220630,95,science,spec:HD142331,FELH0600~holo4_003,37.810374,30.0,238.600995,-8.620363,141.191433
20447,2022063000096,AT_O_20220630_000096,20220630,96,science,spec:HD142331,FELH0600~holo4_003,37.67315,30.0,238.601062,-8.620312,141.191671
20461,2022063000110,AT_O_20220630_000110,20220630,110,science,spec:HD142331,FELH0600~holo4_003,35.936789,30.0,238.602577,-8.619822,139.344998
20462,2022063000111,AT_O_20220630_000111,20220630,111,science,spec:HD142331,FELH0600~holo4_003,35.803062,30.0,238.602655,-8.619746,139.344909


In [28]:
all_def_science_selected_final[3]

IndexError: list index out of range

In [ ]:
all_def_science_selected_final[4]